## Import de bibliotecas a usar

In [19]:
import pandas as pd
import numpy as np
import re

## Definición de función para procesar el dataset proporcionado y limpiarlo

In [20]:
def procesar_cualquier_hoja(df_hoja, nombre_hoja):
    # 1. Limpieza básica
    df = df_hoja.dropna(how='all').dropna(axis=1, how='all')
    
    # 2. Usar las primeras 3 columnas para ensayos
    if len(df.columns) >= 3:
        col_ensayo = df.columns[0]
        col_material = df.columns[1]
        col_resultado = df.columns[2]
    else:
        return None
    
    # 3. BUSCAR FCK Y ASENTAMIENTO - FORMATO UNIFORME
    fck = None
    asentamiento = None
    modulo_finura = None
    
    # Buscar en todo el DataFrame
    for row_idx in range(len(df)):
        for col_idx in range(len(df.columns)):
            try:
                celda_valor = df.iloc[row_idx, col_idx]
                if pd.notna(celda_valor):
                    celda_str = str(celda_valor).strip()
                    
                    # Buscar Fck - Mismo formato que asentamiento
                    if 'kg /' in celda_str and any(c.isdigit() for c in celda_str):
                        # Verificar que sea un valor de Fck típico y que la cadena no sea muy larga
                        if len(celda_str) < 20:
                            numeros = re.findall(r'\d+', celda_str)
                            if numeros:
                                fck = numeros[0]
                    
                    # Buscar Asentamiento - Mantener el formato actual que funciona
                    elif 'cm' in celda_str and any(c.isdigit() for c in celda_str):
                        # Excluir celdas que sean solo texto
                        if not celda_str.lower().strip() in ['asentamiento', 'asentamiento:']:
                            # Verificar que sea un valor de asentamiento típico (ej: "10 cm", "8-10 cm")
                            if len(celda_str) < 20:
                                asentamiento = celda_str        
            except Exception as e:
                continue
    
    # 4. Buscar Módulo de Finura en los datos
    for idx in range(len(df)):
        ensayo_val = df.iloc[idx, 0] if pd.notna(df.iloc[idx, 0]) else None
        
        if pd.notna(ensayo_val) and 'Módulo de finura' in str(ensayo_val):
            resultado_val = df.iloc[idx, 2] if len(df.columns) > 2 and pd.notna(df.iloc[idx, 2]) else None
            if pd.notna(resultado_val):
                resultado_str = str(resultado_val)
                numeros = re.findall(r'[\d,]+\.?[\d]*', resultado_str)
                if numeros:
                    modulo_finura = numeros[0].replace(',', '.')
    
    # 5. Crear DataFrame de ensayos
    try:
        datos_ensayos = []
        for idx in range(len(df)):
            ensayo = df.iloc[idx, 0] if pd.notna(df.iloc[idx, 0]) else None
            material = df.iloc[idx, 1] if len(df.columns) > 1 and pd.notna(df.iloc[idx, 1]) else None
            resultado = df.iloc[idx, 2] if len(df.columns) > 2 and pd.notna(df.iloc[idx, 2]) else None
            
            if pd.notna(ensayo) and pd.notna(material):
                datos_ensayos.append({
                    'ensayo': ensayo,
                    'material': material,
                    'resultado': resultado
                })
        
        df_ensayos = pd.DataFrame(datos_ensayos)
        
        # Filtrar solo ensayos relevantes
        ensayos_a_incluir = ['Densidad', 'Peso unitario', 'Peso unitatario', 'Peso unitário', 'Diámetro máximo']
        df_ensayos = df_ensayos[df_ensayos['ensayo'].apply(
            lambda x: any(ensayo in str(x) for ensayo in ensayos_a_incluir) if pd.notna(x) else False
        )]
        
    except Exception as e:
        return None
    
    # 6. Limpiar resultados y procesar por material
    def limpiar_resultado(valor):
        if pd.isna(valor):
            return None
        valor_str = str(valor).strip()
        numeros = re.findall(r'[\d,]+\.?[\d]*', valor_str)
        if numeros:
            numero_limpio = numeros[0].replace(',', '.')
            try:
                return float(numero_limpio)
            except:
                return None
        return None
    
    df_ensayos['resultado_limpio'] = df_ensayos['resultado'].apply(limpiar_resultado)
    df_ensayos['material_limpio'] = df_ensayos['material'].apply(
        lambda x: str(x).strip() if pd.notna(x) else "Desconocido"
    )
    
    # 7. Procesar por material
    datos_por_material = {}
    
    for idx, fila in df_ensayos.iterrows():
        material = fila['material_limpio']
        ensayo = fila['ensayo']
        resultado = fila['resultado_limpio']
        
        if material == "Desconocido" or pd.isna(ensayo) or pd.isna(resultado):
            continue
            
        if material not in datos_por_material:
            datos_por_material[material] = {'material': material}
        
        # Mapear ensayos a propiedades
        ensayo_str = str(ensayo)
        if 'Densidad' in ensayo_str:
            datos_por_material[material]['densidad'] = resultado
        elif any(peso in ensayo_str for peso in ['Peso unitario', 'Peso unitatario', 'Peso unitário']):
            if 'suelto' in ensayo_str.lower():
                datos_por_material[material]['peso_suelto'] = resultado
            elif 'compactado' in ensayo_str.lower():
                datos_por_material[material]['peso_compactado'] = resultado
        elif 'Diámetro máximo' in ensayo_str:
            datos_por_material[material]['diametro_maximo'] = resultado
    
    # 8. Convertir a DataFrame final
    if datos_por_material:
        df_final = pd.DataFrame(list(datos_por_material.values()))
        
        # Agregar metadatos
        df_final['hoja_origen'] = nombre_hoja
        df_final['fck'] = fck
        df_final['asentamiento'] = asentamiento
        df_final['modulo_finura'] = modulo_finura
        return df_final
    else:
        return None

## Definición de función usando las correcciones anteriores para procesar hoja a hoja los datos proporcionados

In [ ]:
def procesar_todas_las_hojas_corregido():
    archivo_excel = 'datos_lab.xlsx'
   
    try:
        hojas_disponibles = pd.ExcelFile(archivo_excel).sheet_names
    except Exception as e:
        return None
    
    todas_hojas = []
    hojas_exitosas = 0
    
    for nombre_hoja in hojas_disponibles:
        try:
            # Cargar hoja
            df_hoja = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)
            
            # Procesar hoja
            resultado = procesar_cualquier_hoja(df_hoja, nombre_hoja)
            
            if resultado is not None and len(resultado) > 0:
                todas_hojas.append(resultado)
                hojas_exitosas += 1
                
        except Exception as e:
            continue
    
    # COMBINAR TODOS LOS RESULTADOS
    if todas_hojas:
        df_final = pd.concat(todas_hojas, ignore_index=True)
        
        # REORDENAR COLUMNAS
        columnas_orden = ['material', 'densidad', 'peso_suelto', 'peso_compactado', 
                         'diametro_maximo', 'fck', 'asentamiento', 'modulo_finura', 'hoja_origen']
        
        columnas_finales = [col for col in columnas_orden if col in df_final.columns]
        columnas_restantes = [col for col in df_final.columns if col not in columnas_finales]
        df_final = df_final[columnas_finales + columnas_restantes]       
        return df_final
    else:
        return None

In [22]:
# EJECUTAR PROCESAMIENTO
df_final = procesar_todas_las_hojas_corregido()
df_final.to_csv('datos_lab_mejor.csv', index=False, encoding='utf-8')
df_final

,material,densidad,peso_suelto,peso_compactado,diametro_maximo,fck,asentamiento,modulo_finura,hoja_origen
0,Cemento CPII-- F32 Yguazu,3.010,NaN,NaN,NaN,250,8-10 cm,2.69,Hoja5
1,Arena lavada,2.630,NaN,1.52,NaN,250,8-10 cm,2.69,Hoja5
2,Triturada 5ta,2.807,NaN,1.52,NaN,250,8-10 cm,2.69,Hoja5
3,Triturada 4ta,NaN,NaN,1.65,NaN,250,8-10 cm,2.69,Hoja5
4,"Triturada 4ta, Triturada 5ta",NaN,NaN,NaN,25.4,250,8-10 cm,2.69,Hoja5
...,...,...,...,...,...,...,...,...,...
114,Arena Lavada,1.620,1438.14,NaN,NaN,180,None,2.77,Hoja33
115,Cemento Cecon CP II - F40,3.150,NaN,NaN,NaN,180,None,2.77,Hoja34
116,Triturada 5ta,2.490,1432.50,1555.00,12.7,180,None,2.77,Hoja34
117,Triturada 6ta,2.650,1478.94,1707.48,NaN,180,None,2.77,Hoja34
